In [ ]:
import traceback
import pandas as pd
from scipy.integrate import odeint, solve_ivp
from scipy.interpolate import interp1d, InterpolatedUnivariateSpline, UnivariateSpline, splprep, splev, SmoothBivariateSpline
from scipy.ndimage.filters import uniform_filter1d
import matplotlib.pyplot as plt


%env USING_RUN True

%run /work/greenhouse-simulator-2/greenhouse/adaptive_lighting/adaptive_lighting.ipynb import AdaptiveLighting
%run /work/greenhouse-simulator-2/greenhouse/adaptive_lighting/solarpanel.ipynb import SolarPanel
%run /work/greenhouse-simulator-2/greenhouse/heatpump/heatpump.ipynb import HeatPump
%run /work/greenhouse-simulator-2/greenhouse/fan/fan.ipynb import Fan
%run /work/greenhouse-simulator-2/greenhouse/structure/structure.ipynb import Structure
%run /work/greenhouse-simulator-2/greenhouse/dehumidifier/dehumidifier.ipynb import Dehumidifier
%run /work/greenhouse-simulator-2/crops/sweet_basil.ipynb import SweetBasil


import sys
sys.path.insert(0, '/work/greenhouse-simulator-2/')

from helpers.cost import *
from helpers.types import *
from helpers.conversions import *
from helpers.solar_conversions import *
from greenhouse.greenhouse import *
from greenhouse.adaptive_lighting import *

In [ ]:
class Greenhouse:
    def __init__(self, 
        resample_period: s,
        airflow_mode="CONST:0.1",
        temp_mode="const",
        min_temp=17,
        max_temp=27,
        max_humidity=70,
        block_sunlight=False,
        roof_panel_type="polycarbonate"
    ):
        self.time_period: s = pd.to_timedelta(resample_period).total_seconds()

        self.min_temp: C = min_temp
        self.max_temp: C = max_temp
        self.max_humidity: RH = max_humidity

        # Supported values: humidity_control, light_control, CONST:{float}
        self.airflow_mode: str = airflow_mode

        # Supported values: dynamic or else
        self.temp_mode: str = temp_mode

        # Supported values: True, False
        self.block_sunlight = block_sunlight

        # Supported values: insolight, solarbrite, polycarbonate
        self.roof_panel_type = roof_panel_type

        # Init subsystems
        self.structure = Structure(roof_panel_type=self.roof_panel_type)
        self.dehumidifier = Dehumidifier()
        self.heatpump = HeatPump()
        self.fan = Fan()
        self.crop = SweetBasil(
            time_period=self.time_period, 
            plants_per_barrel=self.structure.plants_per_barrel, 
            barrel_count=self.structure.barrel_count
        )
        self.light = AdaptiveLighting(self.time_period, self.structure, self.crop, roof_panel_type=roof_panel_type)

        # Init register to store previous period's values
        self.prev_period = {
            "humidity_ratio": 0.0085881067, # 70 RH at 1 bar and 25 Celsius
            "temp": 17,
            "CO2_concentration": 410,
        }

        self.prev_airflows_at_t_steps = []


    def run(self, timestamp, data):
        # SECTION 1: Get input from solar radiation
        # Get distribution of irradiance on different panels (solar and non-solar) of the greenhouse (factoring in sun position, tilt and azimuth angles)
        results: [W, W_per_m2] = self.structure.get_irradiance_by_panel_type(timestamp, data.solarradiation)
        solar_power_on_nonsolar_panels, irradiance_on_solar_panels = results

        # Get electrical energy generated and heat irradiated to the greenhouse
        results: [W, W, W_per_m2] = self.light.solarpanel.run(irradiance_on_solar_panels)
        solar_power_generated, solar_power_transmitted, transparency = results
        solar_energy_generated: kWh = J_to_kWh(solar_power_generated * self.time_period)

        # ASSUMPTION: roof and front wall has the same transparency (not true if solar panels are installed only on roof)
        transmitted_irradiance = data.solarradiation * transparency

        # Add energy from non-solar panels and portion of energy transmitted by solar panels to get total heat entering the greenhouse
        heat_transfer_rate_inside: W = solar_power_on_nonsolar_panels + solar_power_transmitted

        if self.block_sunlight:
            heat_transfer_rate_inside = 0
            transmitted_irradiance = 0

        # Calculate energy used for lighting
        PPFD: umol_per_m2_s = irradiance_to_PPFD(transmitted_irradiance)
        PAR_inside: umol_per_m2 = PPFD * self.time_period
        lighting_results = self.light.run(timestamp, PAR_inside, self.structure.coordinates)

        # SECTION 2: Grow plants
        results: [g, int, mol_per_s, mol, mol_per_s, mol] = self.crop.grow(1e-6 * lighting_results["actual_PPFD_umol_per_m2_s"] * self.time_period)
        harvested_weight, harvested_plant_count, CO2_assimilation_rate, CO2_assimilated, H20_evaporation_rate, H2O_evaporated = results

        # SECTION 3: Deal with resulting CO2, water, heat
        # Calculate net energy
        total_energy_used: kWh = lighting_results["energy_used_by_lighting_kWh"] #+ energy_for_dehumidification + energy_for_heating
        total_energy_generated: kWh = solar_energy_generated
        net_energy: kWh = total_energy_generated - total_energy_used

        input_values = {
            "H2O_mass_evaporation_rate": 18 * H20_evaporation_rate, # 18: molar mass of H20
            "CO2_assimilation_rate": CO2_assimilation_rate,
            "ambient_data": data, 
            "power_irradiated": transmitted_irradiance * self.structure.irradiated_area,
            "get_heat_transfer_rate": self.structure.get_heat_transfer_rate,
            "structure_volume": self.structure.volume,
        }

        airflow_results = self.register_airflow(input_values, is_light=lighting_results["target_PAR"] != 0)

        energy_used_by_dehum_J, _ = self.dehumidifier.run(airflow_results["dehum_rate_g_per_s"] * self.time_period)
        
        return {
            "harvested_weight_g": harvested_weight,
            "harvested_plant_count": harvested_plant_count,
            "natural_PPFD": PPFD,
            "total_energy_used_kWh": total_energy_used,
            "total_energy_generated_kWh": total_energy_generated,
            "net_energy_kWh": net_energy,

            "CO2_assimilation_rate_umol_per_s": CO2_assimilated / self.time_period * 1e6,

            **lighting_results,
            **airflow_results,
            
            "ambient_humidity": data.humidity,
            "ambient_temp": data.temp,
            "ambient_light_umol_per_m2_s": data.solarradiation * 2.1,

            "energy_used_by_fan_J": self.fan.get_energy_usage(airflow_results["airflow_m3_per_s"] * self.time_period),
            "energy_used_by_heating_J": self.heatpump.get_energy_usage(airflow_results["heating_rate_J_per_s"] * self.time_period),
            "energy_used_by_dehum_J": energy_used_by_dehum_J,
        }

    def register_airflow(self, input_values, is_light):
        # Set up initial values for odeint
        init_values = [
            self.prev_period["humidity_ratio"], 
            self.prev_period["temp"], 
            self.prev_period["CO2_concentration"]
        ]

        # List of dicts where the t values and control signals of each integration steps can be stored
        control_register = []

        # Set up timesteps for odeint (we are interested only in the last one)
        t_steps = np.linspace(0, self.time_period, 15)
        t_max = t_steps[-1]

        # Set up control signal config
        control_config = {
            "airflow_mode": self.airflow_mode,
            "temp_mode": self.temp_mode,
            "min_temp": self.min_temp,
            "max_temp": self.max_temp,
            "max_humidity": self.max_humidity,
            "is_light": is_light,
        }

        # Solve diff equations
        results = solve_ivp(
            airflow_model, 
            (0, t_max,), 
            init_values, 
            t_eval=t_steps, 
            method="BDF", 
            dense_output=True, 
            args=(t_max, input_values, control_register, self.prev_airflows_at_t_steps, control_config)
        )

        new_humidity_ratio, new_temp, new_CO2_concentration = results["y"].T[-1]

        pressure: Pa = 101325 # TODO: get this from weather data
        new_humidity: RH = psychrolib.GetRelHumFromHumRatio(new_temp, new_humidity_ratio, pressure) * 100

        # Since solve_ivp calculates the steps randomly, needs to be sorted based on t
        control_register = sorted(control_register, key=lambda x: x["t"])

        # Set up and call interpolation function to get control values at t_max
        control_results = {}
        control_t_values = [x["t"] for x in control_register]
        for control_type in control_register[0]:
            control_results[control_type] = control_register[-1][control_type]

        # Set new values as starting values for next period
        self.prev_period["humidity_ratio"] = new_humidity_ratio
        self.prev_period["temp"] = new_temp
        self.prev_period["CO2_concentration"] = new_CO2_concentration

        return {
            "humidity": new_humidity,
            "humidity_ratio": new_humidity_ratio,
            "temp": new_temp,
            "CO2_concentration": new_CO2_concentration,
            **control_results,
        }



## Execute

In [ ]:
if not os.environ.get("USING_RUN"):
    is_long = False
    if is_long:
        resample_period = "60min"
        df = get_weather_data(date_from="2020-01-01", date_to="2020-12-31", resample_period=resample_period)
    else:
        resample_period = "15min"
        df = get_weather_data(date_from="2020-07-01", date_to="2020-07-02", resample_period=resample_period)

In [ ]:
if not os.environ.get("USING_RUN"):
    time_period = pd.to_timedelta(resample_period).total_seconds()

    greenhouse = Greenhouse(
        time_period = time_period,
    )

    for timestamp, row in df.iterrows():
        try:
            results = greenhouse.run(timestamp, row)
        except Exception as e:
            traceback.print_exc()
            break

        for col in results.keys():
            df.loc[timestamp, col] = results[col]

## Visualize

In [ ]:
if not os.environ.get("USING_RUN"):
    plot_multiline_dual_y(
        df.resample('D').mean() if is_long else df, 
        ["airflow_m3_per_s"],
        width=900, 
        height=235, 
        title="Airflow", 
        y_labels=["m3 / s", "g / s"], 
        legend_label="", 
        date_format="",
        colors=["blue", "green", "orange"]
    )

In [ ]:
if not os.environ.get("USING_RUN"):
    plot_multiline_dual_y(
        df.resample('D').mean() if is_long else df, 
        ["humidity", "ambient_humidity", "ambient_humidity_at_inside_temp_RH"],
        ["dehum_rate_g_per_s"],
        width=900, 
        height=235, 
        title="Humidity", 
        y_labels=["RH %", "g / s"], 
        legend_label="", 
        date_format="",
        colors=["blue", "lightgreen", "green", "orange"],
        left_axis_target_range=[40,70]
    )

In [ ]:
if not os.environ.get("USING_RUN"):
    plot_multiline_dual_y(
        df.resample('D').mean() if is_long else df, 
        ["temp", "ambient_temp"],
        ["heating_rate_J_per_s"],
        width=900, 
        height=235, 
        title="Temperature", 
        y_labels=["C", "J / s"], 
        legend_label="", 
        date_format="",
        colors=["blue", "green", "orange"],
        left_axis_target_range=[17,27]
    )

In [ ]:
if not os.environ.get("USING_RUN"):
    plot_multiline_dual_y(
        df.resample('D').mean() if is_long else df, 
        ["CO2_concentration"],
        ["CO2_release_rate_mol_per_s"],
        width=900, 
        height=235, 
        title="CO2", 
        y_labels=["ppm", "mol / s"], 
        legend_label="", 
        date_format="",
        colors=["blue", "green"]
    )

### Calculate energy usage

In [ ]:
if not os.environ.get("USING_RUN"):
    electricity_cost: EUR_per_kWh = 0.145

    # Fan
    auc_airflow: m3 = np.trapz(df["airflow_m3_per_s"], dx=time_period)
    energy_used_by_fan: J = greenhouse.fan.get_energy_usage(auc_airflow)

    # Heating
    auc_heating: J = np.trapz(df["heating_rate_J_per_s"], dx=time_period)
    energy_used_by_heating: J = greenhouse.heatpump.get_energy_usage(auc_heating)

    # Dehumidification
    auc_dehum: g = np.trapz(df["dehum_rate_g_per_s"], dx=time_period)
    energy_used_by_dehum, _ = greenhouse.dehumidifier.run(auc_dehum)

    # Lighting
    energy_used_by_lighting: J = df['energy_used_by_lighting_J'].sum()

    total_energy: J = energy_used_by_fan + energy_used_by_heating + energy_used_by_dehum + energy_used_by_lighting

    print(f"Total energy used: {round(J_to_kWh(total_energy), 2)} kWh, costs ~ {round(J_to_kWh(total_energy) * electricity_cost, 2)} EUR")

In [ ]:
if not os.environ.get("USING_RUN"):
    for i, row in df.iterrows():
        df.at[i, "energy_used_by_fan_J"]      = greenhouse.fan.get_energy_usage(row["airflow_m3_per_s"] * time_period)
        df.at[i, "energy_used_by_heating_J"]  = greenhouse.heatpump.get_energy_usage(row["heating_rate_J_per_s"] * time_period)
        df.at[i, "energy_used_by_dehum_J"], _ = greenhouse.dehumidifier.run(row["dehum_rate_g_per_s"] * time_period)

    plot_stacked_area(
        df, 
        ["energy_used_by_fan_J", "energy_used_by_heating_J", "energy_used_by_dehum_J", "energy_used_by_lighting_J"], 
        width=900,
        y_label="Energy used (J)",
        legend_label=""
    )

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2f5dc715-67f7-4c8c-98f7-a87b736d3338' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>